In [65]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
import seaborn as sns
import pandas as pd
import numpy as np 
import scipy.stats as stats
data_df = pd.read_csv('../../datasets/AdultCensusIncomeForecast_train.csv')
data_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [66]:
'''
| age            | int64     | 나이           | max bound 이상은 max bound 처리 
| fnlwgt         | int64     | 가중치값       | 현재 표본에서 전체 인구를 추정할 때 
| education_num  | int64     | 교육 수준      | education과 같은 
| capital_gain   | int64     | 자본 이득      | 0인 값에 대한 쏠림 
| capital_loss   | int64     | 자본 손실      | 0인 값에 대한 쏠림   
| hours_per_week | int64     | 주당 근무 시간 | 주당 근무 시간 분포 알수 
| workclass      | object    | 직업분류       | 8가지 정도의 직업군 
| education      | object    | 교육 수준      | education_num과 같은 
| marital_status | object    | 결혼 상태      | 7가지 결혼 
| occupation     | object    | 직업           | 14가지 직업군   
| relationship   | object    | 가족 관계      | 6가지 관계 
| race           | object    | 인종           | 5가지 인종 
| gender         | object    | 성별           | 두가지 성별 
| native_country | object    | 출신 국가      | 41개 국가 미국 출신이 
| income         | object    | 소득 수준      | <=50K 또는 >50K 의 두가지 
'''

'\n| age            | int64     | 나이           | max bound 이상은 max bound 처리 \n| fnlwgt         | int64     | 가중치값       | 현재 표본에서 전체 인구를 추정할 때 \n| education_num  | int64     | 교육 수준      | education과 같은 \n| capital_gain   | int64     | 자본 이득      | 0인 값에 대한 쏠림 \n| capital_loss   | int64     | 자본 손실      | 0인 값에 대한 쏠림   \n| hours_per_week | int64     | 주당 근무 시간 | 주당 근무 시간 분포 알수 \n| workclass      | object    | 직업분류       | 8가지 정도의 직업군 \n| education      | object    | 교육 수준      | education_num과 같은 \n| marital_status | object    | 결혼 상태      | 7가지 결혼 \n| occupation     | object    | 직업           | 14가지 직업군   \n| relationship   | object    | 가족 관계      | 6가지 관계 \n| race           | object    | 인종           | 5가지 인종 \n| gender         | object    | 성별           | 두가지 성별 \n| native_country | object    | 출신 국가      | 41개 국가 미국 출신이 \n| income         | object    | 소득 수준      | <=50K 또는 >50K 의 두가지 \n'

In [67]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   gender          32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


### 예비 가설 
- 분석 내용 : 성별에 따른 나이 차이가 있는가?
  - 귀무 가설 : 나이에 차이가 없다
  - 대립 가설 : 나이에 차이가 있다
- 결론 도출 : 대립 가설 성립 (pvalue : 3.60e-56 < 0.05)
  - 성별에 따라 나이 분포에 통계적으로 유의한 차이가 있다

### anderson : 정규 분포 여부 확인
- gender : male, female
- age 정규 분포 (0.787 > 0.05)

In [68]:
data_df['gender'].value_counts()

gender
Male      21790
Female    10771
Name: count, dtype: int64

In [69]:
condition_dual = 'gender in ("Male", "Female")'
duale_df = data_df.query(condition_dual)
len(duale_df)

32561

In [70]:
results = stats.anderson(duale_df['age'].dropna())
results

AndersonResult(statistic=238.08301913675677, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=38.58164675532078, scale=13.640432553581341)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [71]:
results.significance_level, results.critical_values

(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

### 등분산 검증 : levene
- pvalue=9.10e-10 < 0.05 이분산

In [72]:
condition_male = 'gender == "Male"'
condition_female = 'gender == "Female"'

In [73]:
male_df = data_df.query(condition_male)
female_df = data_df.query(condition_female)

len(male_df), len(female_df)

(21790, 10771)

In [74]:
stats.levene(male_df['age'], female_df['age'])


LeveneResult(statistic=37.5294932933936, pvalue=9.107127745249435e-10)

### 분포 확인 : 범주형 2개인 경우 
- T Test (ttest_ind) (T value 라는 값이 있는데 확인 필요) : 등분산 일 때 equal_var=True 주어야함.

In [75]:
test_list = [male_df['age'], female_df['age']]
stats.ttest_ind(*test_list, equal_var=False)

TtestResult(statistic=15.838876900662953, pvalue=3.603268494435076e-56, df=20583.32674466559)

### 예비 가설 
- 분석 내용 : 출신국가에 따른 자본 이득 차이가 있는가?
  - 귀무 가설 : 출신국가에 따른 자본 이득 차이가 없다
  - 대립 가설 : 출신국가에 따른 자본 이득 차이가 있다
- 결론 도출 : 대립 가설 성립 (pvalue : 2.09e-05 < 0.05)
  - 출신국가에 따른 자본 이득 분포에 통계적으로 유의한 차이가 있다

In [76]:
data_df['native_country'].value_counts()

native_country
United-States                 29170
Mexico                          643
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
France                           29
Greece       

### anderson : 정규 분포 여부 확인
- native_country : United-States, Mexico, Philippines
- capital_gain 정규 분포 (0.787 > 0.05)

In [77]:
condition_triple = 'native_country in ("United-States", "Mexico", "Philippines")'
triple_df = data_df.query(condition_triple).dropna()
len(triple_df)

28302

In [78]:
results = stats.anderson(triple_df['capital_gain'].dropna())
results

AndersonResult(statistic=9054.000797734669, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=1103.7755282312205, scale=7458.63102556717)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [79]:
results.significance_level, results.critical_values


(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

### 등분산 검증 : levene
- pvalue=0.04 < 0.05 이분산

In [80]:
condition_usa = 'native_country == "United-States"'
condition_mexico = 'native_country == "Mexico"'
condition_philippines = 'native_country == "Philippines"'

In [81]:
df_usa = data_df.query(condition_usa).dropna()
df_mexico = data_df.query(condition_mexico).dropna()
df_philippines = data_df.query(condition_philippines).dropna()

len(df_usa), len(df_mexico), len(df_philippines)

(27504, 610, 188)

In [82]:
stats.levene(df_usa['capital_gain'], df_mexico['capital_gain'], df_philippines['capital_gain'])

LeveneResult(statistic=3.1095982456468545, pvalue=0.04463412394675287)

### 범주형 3개 이상인 경우 
- ANOVA (f_oneway) : 등분산
- kruskal Test (kruskal) : 이분산

In [83]:
test_list = [df_usa['capital_gain'], df_mexico['capital_gain'], df_philippines['capital_gain']]
stats.kruskal(*test_list)

KruskalResult(statistic=21.549641126972606, pvalue=2.09196727150439e-05)

### 예비 가설 
- 분석 내용 : 출신국가에 따른 가중치(fnlwgt) 차이가 있는가?
  - 귀무 가설 : 출신국가에 따른 가중치(fnlwgt) 차이가 없다
  - 대립 가설 : 출신국가에 따른 가중치(fnlwgt) 차이가 있다
- 결론 도출 : 대립 가설 성립 (pvalue : 6.45e-121 < 0.05)
  - 출신국가에 따른 가중치(fnlwgt) 분포에 통계적으로 유의한 차이가 있다

### anderson : 정규 분포 여부 확인
- native_country : United-States, Mexico, Philippines
- fnlwgt 정규 분포 (0.787 > 0.05)

In [84]:
results = stats.anderson(triple_df['fnlwgt'].dropna())
results

AndersonResult(statistic=330.83131717534343, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=189134.3291286835, scale=105897.99357751115)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [85]:
results.significance_level, results.critical_values


(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

### 등분산 검증 : levene
- pvalue=0.00 < 0.05 이분산

In [86]:
stats.levene(df_usa['fnlwgt'], df_mexico['fnlwgt'], df_philippines['fnlwgt'])


LeveneResult(statistic=7.555984325994428, pvalue=0.0005240268912846971)

### 범주형 3개 이상인 경우 
- ANOVA (f_oneway) : 등분산
- kruskal Test (kruskal) : 이분산

In [87]:
test_list = [df_usa['fnlwgt'], df_mexico['fnlwgt'], df_philippines['fnlwgt']]
stats.kruskal(*test_list)


KruskalResult(statistic=553.4957569029987, pvalue=6.4554052349080445e-121)

### 예비 가설 
- 분석 내용 : 결혼관계에 따른 가중치(fnlwgt) 차이가 있는가?
  - 귀무 가설 : 결혼관계에 따른 가중치(fnlwgt) 차이가 없다
  - 대립 가설 : 결혼관계에 따른 가중치(fnlwgt) 차이가 있다
- 결론 도출 : 대립 가설 성립 (pvalue : 7.43e-05 < 0.05)
  - 결혼관계에 따른 가중치(fnlwgt) 분포에 통계적으로 유의한 차이가 있다

In [88]:
data_df['relationship'].value_counts()

relationship
Husband           13193
Not-in-family      8305
Own-child          5068
Unmarried          3446
Wife               1568
Other-relative      981
Name: count, dtype: int64

### anderson : 정규 분포 여부 확인
- relationship : Husband, Not-in-family, Wife
- fnlwgt 정규 분포 (0.787 > 0.05)

In [89]:
condition_triple = 'relationship in ("Husband", "Not-in-family", "Wife")'
triple_df = data_df.query(condition_triple).dropna()
len(triple_df)

21595

In [90]:
results = stats.anderson(triple_df['fnlwgt'].dropna())
results

AndersonResult(statistic=241.22408438088314, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=188211.22065292893, scale=105178.28085411784)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

In [91]:
results.significance_level, results.critical_values


(array([15. , 10. ,  5. ,  2.5,  1. ]),
 array([0.576, 0.656, 0.787, 0.918, 1.092]))

### 등분산 검증 : levene
- pvalue=0.00 < 0.05 이분산

In [92]:
condition_usa = 'relationship == "Husband"'
condition_mexico = 'relationship == "Not-in-family"'
condition_philippines = 'relationship == "Wife"'

In [93]:
df_usa = data_df.query(condition_usa).dropna()
df_mexico = data_df.query(condition_mexico).dropna()
df_philippines = data_df.query(condition_philippines).dropna()

len(df_usa), len(df_mexico), len(df_philippines)

(12463, 7726, 1406)

In [94]:
test_list = [df_usa['fnlwgt'], df_mexico['fnlwgt'], df_philippines['fnlwgt']]

In [95]:
stats.levene(*test_list)

LeveneResult(statistic=8.170037640455734, pvalue=0.00028388316396871894)

### 범주형 3개 이상인 경우 
- ANOVA (f_oneway) : 등분산
- kruskal Test (kruskal) : 이분산

In [96]:
stats.kruskal(*test_list)

KruskalResult(statistic=19.01230647242693, pvalue=7.439266302722759e-05)